In [111]:
import xml.etree.ElementTree as ET

def xml_to_dict(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()

    def recurse(node):
        if len(node) == 0:
            return node.text
        else:
            result = {}
            for child in node:
                if child.tag not in result:
                    result[child.tag] = recurse(child)
                else:
                    if type(result[child.tag]) is list:
                        result[child.tag].append(recurse(child))
                    else:
                        result[child.tag] = [result[child.tag], recurse(child)]
            return result

    return {root.tag: recurse(root)}


In [112]:
result_dict = xml_to_dict("./pdbxml.xml")

print(list(result_dict.keys()))

['DatabaseModel']


In [113]:
import re

def sql_to_dict(sql_text):
    # Wyszukuje wszystkie definicje tabel
    tables = re.findall(    r"CREATE TABLE (.*?);", sql_text, re.DOTALL)
    references = re.findall(r"ALTER TABLE (.*?);", sql_text, re.DOTALL)


    result = {}
    for table in tables:
        # Wyszukuje nazwę tabeli i definicję
        match = re.match(r"(\w*)\s*(.*)", table, re.DOTALL)
        if match:
            table_name = match.group(1)
            table_definition = match.group(2)
            result[table_name] = f'CREATE TABLE {table_name} {table_definition};'
    for ref in references:
        match = re.match(r"(\w*)\s*(.*)", ref, re.DOTALL)
        if match:
            table_name = match.group(1)
            ref_definition = match.group(2)
            result[table_name] += f'\nALTER TABLE {table_name} {ref_definition};'
    return result

with open('./szkola.sql', 'rt', encoding='utf-8') as f:
    d = sql_to_dict(f.read())

In [114]:
def filter_checks(string:str):
    return '\n'.join([ line for line in string.split('\n') if 'CHECK' not in line])

In [115]:
def extract_sql_entities(sql_text):
    pattern = r"(-- [^\n]+)\n(CREATE OR ALTER (?:VIEW|FUNCTION|PROCEDURE) [^\n]+) AS\n([\s\S]+?)\nGO"

    matches = re.findall(pattern, sql_text)

    extracted_data = []
    for match in matches:
        comment, create_statement, body = match
        entity_name = create_statement.split()[4]  # Assumes the name is the 5th word
        entity_type = "procedure" if "PROCEDURE" in create_statement else "function" if "FUNCTION" in create_statement else "view"
        
        extracted_data.append({
            "name": entity_name,
            "type": entity_type,
            "create": create_statement + " AS\n" + body,
            "comment": comment.lstrip('-- ').strip()

        })

    return extracted_data


def extract_actions(sql_string):
    # Regular expression to match the pattern, including full names with schema
    pattern = r"(?:--(.*?)\n)?\b(CREATE(?:\s+OR\s+ALTER)?\s+(FUNCTION|PROCEDURE|TRIGGER)\s+([\w.]+).*?;\s*GO)"
    matches = re.findall(pattern, sql_string, re.DOTALL | re.IGNORECASE)

    result = []
    for match in matches:
        comment = match[0].strip() if match[0] else ''
        result.append({
            "name": match[3],
            "type": match[2].lower(),
            "comment": comment,
            "create": match[1].strip()
        })

    return result

# Test the function with an example SQL string including schema names
example_sql_with_full_names = """
-- This is a function in dbo schema
CREATE OR ALTER FUNCTION dbo.calculate_something()
RETURNS INT AS $$
BEGIN
    -- Function body goes here
END;
GO

-- Procedure in the dbo schema
CREATE OR ALTER PROCEDURE dbo.process_data()
LANGUAGE SQL AS $$
BEGIN
    -- Procedure body goes here
END;
GO
"""
extract_actions(example_sql_with_full_names)

[{'name': 'dbo.calculate_something',
  'type': 'function',
  'comment': 'This is a function in dbo schema',
  'create': 'CREATE OR ALTER FUNCTION dbo.calculate_something()\nRETURNS INT AS $$\nBEGIN\n    -- Function body goes here\nEND;\nGO'},
 {'name': 'dbo.process_data',
  'type': 'procedure',
  'comment': 'Procedure in the dbo schema',
  'create': 'CREATE OR ALTER PROCEDURE dbo.process_data()\nLANGUAGE SQL AS $$\nBEGIN\n    -- Procedure body goes here\nEND;\nGO'}]

In [122]:
file = open('./output.org', 'w', encoding='utf-8')
print(r'#+LATEX_HEADER: \usepackage[a4paper, left=2.25cm, right=2.25cm, top=1.25cm, bottom=1.25cm]{geometry}', file=file)

print(r'#+LATEX_HEADER: \usepackage{lmodern}', file=file)
print(r'#+author: Piotr Karamon, Kyrylo Iakymenko, Joanna Konieczny', file=file)
print(r'#+title: Dokumentacja', file=file)
print(r'#+date: ', file=file)
print(file=file)

with open('./funkcje_system.org', 'rt', encoding='utf-8') as f:
    print(f.read(),file=file)

print('* Tabele', file=file)
for table in result_dict['DatabaseModel']['Tables']['Table']:
    print(f'** {table["Name"]}', file=file)
    print(f'{table["Description"]}', file=file)

    columns = table["Columns"]["Column"]
    if isinstance(columns, dict):
        columns = [columns]
        
    for column in columns:
        description = '- ' + column["Description"] if column["Description"] is not None else ""
        print(f"+ ={column['Name']}= {description}", file=file)

    table_checks = table['TableChecks']
    table_df = filter_checks(d[table["Name"]])
    print(f'#+begin_src sql\n{table_df}\n#+end_src', file=file)

    if table_checks is None:
        continue
    print('Warunki integralnościowe:\n\n', file=file)
    for table_check_key in table_checks:
        table_check = table_checks[table_check_key]
        if isinstance(table_check, dict):
            table_check = [table_check]
        
        for tb in table_check:
            print(f'+ ={tb["Name"]}=', file=file)
            if tb["Description"] is not None:
                print(f'\n    {tb["Description"]}', file=file)

            print(f'    #+begin_src sql\nCONSTRAINT {tb["Name"]} CHECK\n({tb["CheckExpression"]})\n   #+end_src', file=file)

print('* Widoki', file=file)
# Example usage with the previous SQL content
with open('./views.sql', 'rt', encoding='utf-8') as f:
    extracted_entities = extract_sql_entities(f.read())

    for entity in extracted_entities:
        print(f'** ={entity["name"]}=', file=file)
        print(f'{entity["comment"]}', file=file)
        print(f"#+begin_src sql\n{entity['create']}\n#+end_src", file=file)
print('* Funkcje i procedury', file=file)
with open('./fp.sql', 'rt', encoding='utf-8') as f:
    extracted_fs = extract_actions(f.read())
    for f in extracted_fs:
        print(f'** ={f["name"]}=', file=file)
        print(f'{f["comment"]}', file=file)
        print(f"#+begin_src sql\n{f['create']}\n#+end_src", file=file)

print('* Triggery', file=file)
with open('./triggers.sql', 'rt', encoding='utf-8') as f:
    for action in extract_actions(f.read()):
        print(f'** ={action["name"]}=', file=file)
        print(f'{action["comment"]}', file=file)
        print(f"#+begin_src sql\n{action['create']}\n#+end_src", file=file)

print('* Indeksy', file=file)
with open('./indexes.sql', 'rt', encoding='utf-8') as f:
    extracted_indexes = extract_indexes_from_sql(f.read())
    for index in extracted_indexes:
        print(f'** ={index["name"]}=', file=file)
        print(f'{index["comment"]}', file=file)
        print(f"#+begin_src sql\n{index['create']}\n#+end_src", file=file)

print('* Role', file=file)
with open('./roles.sql', 'rt', encoding='utf-8') as f:
    roles = extract_roles_with_comments_permissions(f.read())
    for role in roles:
        print(f'** ={role["name"]}=', file=file)
        print(f'{role["comment"]}', file=file)
        print(f"#+begin_src sql\n{role['create']}\n#+end_src", file=file)



file.close()


In [119]:
import re

def clean_sql_script(sql_script):
    pattern = r"(CREATE TABLE\s+[\s\S]+?;)|(ALTER TABLE\s+[\s\S]+?;)"
    
    # Find all matches in the script
    matches = re.findall(pattern, sql_script, re.IGNORECASE)
    
    # Extract matched strings and concatenate them
    cleaned_script = "\n".join(match[0] or match[1] for match in matches)
    
    return cleaned_script

sql_script = """-- Your SQL script here..."""
with open('./szkola.sql','rt', encoding='utf-8' ) as f:
    cleaned_script = clean_sql_script(f.read())
    with open('./cleaned.sql', 'wt', encoding='utf-8') as f:
        f.write(cleaned_script)
